In [98]:
import requests

for _ in range(10):
    requests.get('http://127.0.0.1:5000/suggestion/z/limit/5').text

In [3]:
import sqlite3

conn = sqlite3.Connection('stable/data/data.db')
cur = conn.cursor()
cur.execute("select id, clean_title, count from articles order by count desc")
articles = cur.fetchall()
articles = list(sorted(
    articles, key=lambda x: x[1]
))

Binary exact match search

In [34]:
def bin_search(array, string):
    left = 0
    right = len(array)
    mid = ((right - left) // 2) + left

    while left < right:
        if string == array[mid][1]:
            return mid
        elif string < array[mid][1]:
            right = mid
            mid = ((right - left) // 2) + left
        else:
            left = mid + 1
            mid = ((right - left) // 2) + left
    return None


import random

for _ in range(1000000):
    to_find = random.randint(0, len(articles) - 1)
    
    idx = bin_search(articles, articles[to_find][1])
    assert articles[idx][1] == articles[to_find][1], (to_find, articles[to_find], idx, articles[idx])

Binary prefix search with full slice sorting. 3 seconds for 2000 searches.

In [86]:
def bin_prefix_search(array, prefix, limit):
    left = 0
    right = len(array)
    mid = ((right - left) // 2) + left
    size = len(prefix)

    found = False
    while left < right:
        if prefix == array[mid][1][: size]:
            found = True
            break
        elif prefix < array[mid][1][: size]:
            right = mid
            mid = ((right - left) // 2) + left
        else:
            left = mid + 1
            mid = ((right - left) // 2) + left

    if not found:
        return None
            
    left = mid
    while 0 < left - 1 and prefix == array[left - 1][1][: size]:
        left -= 1
        
    # Don't add one to comparisons with right if using slices because it is exclusive
    right = mid
    while right + 1 <= len(array) and prefix == array[right][1][: size]:
        right += 1

    return list(
        sorted(
            array[left : right],
            key=lambda x: x[2],
            reverse=True
        )[: limit]
    )


import random

for _ in range(2000):
    prefix_size = random.randint(1, 10)
    idx = random.randint(0, len(articles))
    prefix = articles[idx][1][: prefix_size]

    for result in bin_prefix_search(articles, prefix, 5):
        assert result[1][: min(len(prefix), prefix_size)] == prefix, (prefix, result[1], prefix_size)



In [17]:
from tree import Item, BTree

tree = BTree()

for i in range(len(articles)):
    id, string, count = articles[i]
    if string.lower().strip() == 'tokyo':
        print(string)
    tree.insert(Item(id, string, count))



In [4]:
tree.print()

In [3]:
print(tree.find('japan'))

NameError: name 'tree' is not defined